In [4]:
from sqlalchemy import create_engine
import pandas as pd
import pyodbc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer

In [5]:
########### CONNECT TO ThalesStockPredictor SQLServer DB

# Define connection string
connStr = 'mssql+pyodbc://@MSI/ThalesStockPredictor?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'

# Create SQLAlchemy engine
engine = create_engine(connStr)

# Define the query
query = """
SELECT * FROM vw_COMBINED_MODEL
ORDER BY FK_DT_Date desc
"""

# Execute the query and assign the result to a pandas DataFrame
Model_Data = pd.read_sql(query, engine)

# Close the SQL Server Connection
engine.dispose()

In [6]:
# View the data
Model_Data

,FK_DT_Date,THA_Open,THA_High,THA_Low,THA_Close,THA_Adj_Close,THA_Volume,THA_NextDay_Close,SPI_Open,SPI_High,...,SPI_SMA,SPI_UpperBand,SPI_LowerBand,FRA_SMA,FRA_UpperBand,FRA_LowerBand,THA_OBV,EUR_OBV,SPI_OBV,FRA_OBV
0,2024-03-05,145.000000,151.050003,143.350006,150.899994,150.899994,669756.0,NaN,5110.520020,5114.540039,...,5038.216455,5149.478140,4926.954770,7813.302490,8071.131062,7555.473919,21392734.0,2.318764e+10,1.295084e+12,8.148700e+09
1,2024-03-04,136.000000,138.600006,135.850006,138.350006,138.350006,351633.0,150.899994,5130.990234,5149.669922,...,5032.122949,5150.409361,4913.836537,7798.610010,8061.207936,7536.012083,20722978.0,2.318764e+10,1.296771e+12,8.148700e+09
2,2024-03-01,137.500000,138.250000,135.399994,135.399994,135.399994,236008.0,138.350006,5098.509766,5140.330078,...,5023.505933,5136.468663,4910.543202,7780.287500,8047.620766,7512.954234,20371345.0,2.318886e+10,1.301530e+12,8.104148e+09
3,2024-02-29,137.250000,137.300003,135.699997,137.050003,137.050003,984695.0,135.399994,5085.359863,5104.990234,...,5011.961426,5123.232730,4900.690122,7763.191992,8032.810869,7493.573116,20607353.0,2.310308e+10,1.296782e+12,8.053039e+09
4,2024-02-28,137.000000,137.149994,135.850006,136.600006,136.600006,186668.0,137.050003,5067.200195,5077.370117,...,4999.430420,5126.106344,4872.754496,7746.257983,8014.985481,7477.530486,19622658.0,2.300034e+10,1.291562e+12,8.161566e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6211,2000-01-07,33.500000,35.119999,33.349998,35.049999,19.815237,506662.0,35.200001,1403.449951,1441.469971,...,1420.333984,1472.471683,1368.196286,5611.762012,5993.606240,5229.917784,-180815.0,-8.448290e+07,2.394000e+09,0.000000e+00
6212,2000-01-06,33.000000,33.400002,32.599998,33.160000,18.746742,178098.0,35.049999,1402.109985,1411.900024,...,1415.049988,1468.714715,1361.385261,5629.800049,6060.764238,5198.835860,-687477.0,-1.405418e+08,1.168800e+09,0.000000e+00
6213,2000-01-05,33.599998,34.599998,32.000000,32.000000,18.090946,429936.0,33.160000,1399.420044,1413.270020,...,1418.916667,1481.853345,1355.979989,5689.696777,6128.436294,5250.957260,-865575.0,-9.936120e+07,7.650003e+07,0.000000e+00
6214,2000-01-04,35.950001,35.950001,33.099998,33.750000,19.080296,435639.0,32.000000,1455.219971,1455.219971,...,1427.320007,1506.233021,1348.406994,5794.695068,6141.672504,5447.717633,-435639.0,-4.667840e+07,-1.009000e+09,0.000000e+00


In [7]:
# Copy original data frame for normalization
Model_ML = Model_Data.copy()

In [8]:
Model_ML

,FK_DT_Date,THA_Open,THA_High,THA_Low,THA_Close,THA_Adj_Close,THA_Volume,THA_NextDay_Close,SPI_Open,SPI_High,...,SPI_SMA,SPI_UpperBand,SPI_LowerBand,FRA_SMA,FRA_UpperBand,FRA_LowerBand,THA_OBV,EUR_OBV,SPI_OBV,FRA_OBV
0,2024-03-05,145.000000,151.050003,143.350006,150.899994,150.899994,669756.0,NaN,5110.520020,5114.540039,...,5038.216455,5149.478140,4926.954770,7813.302490,8071.131062,7555.473919,21392734.0,2.318764e+10,1.295084e+12,8.148700e+09
1,2024-03-04,136.000000,138.600006,135.850006,138.350006,138.350006,351633.0,150.899994,5130.990234,5149.669922,...,5032.122949,5150.409361,4913.836537,7798.610010,8061.207936,7536.012083,20722978.0,2.318764e+10,1.296771e+12,8.148700e+09
2,2024-03-01,137.500000,138.250000,135.399994,135.399994,135.399994,236008.0,138.350006,5098.509766,5140.330078,...,5023.505933,5136.468663,4910.543202,7780.287500,8047.620766,7512.954234,20371345.0,2.318886e+10,1.301530e+12,8.104148e+09
3,2024-02-29,137.250000,137.300003,135.699997,137.050003,137.050003,984695.0,135.399994,5085.359863,5104.990234,...,5011.961426,5123.232730,4900.690122,7763.191992,8032.810869,7493.573116,20607353.0,2.310308e+10,1.296782e+12,8.053039e+09
4,2024-02-28,137.000000,137.149994,135.850006,136.600006,136.600006,186668.0,137.050003,5067.200195,5077.370117,...,4999.430420,5126.106344,4872.754496,7746.257983,8014.985481,7477.530486,19622658.0,2.300034e+10,1.291562e+12,8.161566e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6211,2000-01-07,33.500000,35.119999,33.349998,35.049999,19.815237,506662.0,35.200001,1403.449951,1441.469971,...,1420.333984,1472.471683,1368.196286,5611.762012,5993.606240,5229.917784,-180815.0,-8.448290e+07,2.394000e+09,0.000000e+00
6212,2000-01-06,33.000000,33.400002,32.599998,33.160000,18.746742,178098.0,35.049999,1402.109985,1411.900024,...,1415.049988,1468.714715,1361.385261,5629.800049,6060.764238,5198.835860,-687477.0,-1.405418e+08,1.168800e+09,0.000000e+00
6213,2000-01-05,33.599998,34.599998,32.000000,32.000000,18.090946,429936.0,33.160000,1399.420044,1413.270020,...,1418.916667,1481.853345,1355.979989,5689.696777,6128.436294,5250.957260,-865575.0,-9.936120e+07,7.650003e+07,0.000000e+00
6214,2000-01-04,35.950001,35.950001,33.099998,33.750000,19.080296,435639.0,32.000000,1455.219971,1455.219971,...,1427.320007,1506.233021,1348.406994,5794.695068,6141.672504,5447.717633,-435639.0,-4.667840e+07,-1.009000e+09,0.000000e+00


In [9]:
# Convert 'FK_DT_Date' to UNIX Epoch (numeric date)
Model_ML['FK_DT_Date'] = pd.to_datetime(Model_ML['FK_DT_Date']).astype('int64') // 10**9

# Remove rows with NA values (equivalent to na.omit in R)
Model_ML.dropna(inplace=True)

# Reset row names (equivalent to removing row names in R)
Model_ML.reset_index(drop=True)

Model_ML

,FK_DT_Date,THA_Open,THA_High,THA_Low,THA_Close,THA_Adj_Close,THA_Volume,THA_NextDay_Close,SPI_Open,SPI_High,...,SPI_SMA,SPI_UpperBand,SPI_LowerBand,FRA_SMA,FRA_UpperBand,FRA_LowerBand,THA_OBV,EUR_OBV,SPI_OBV,FRA_OBV
1,1709510400,136.000000,138.600006,135.850006,138.350006,138.350006,351633.0,150.899994,5130.990234,5149.669922,...,5032.122949,5150.409361,4913.836537,7798.610010,8061.207936,7536.012083,20722978.0,2.318764e+10,1.296771e+12,8.148700e+09
4,1709078400,137.000000,137.149994,135.850006,136.600006,136.600006,186668.0,137.050003,5067.200195,5077.370117,...,4999.430420,5126.106344,4872.754496,7746.257983,8014.985481,7477.530486,19622658.0,2.300034e+10,1.291562e+12,8.161566e+09
5,1708992000,138.899994,139.100006,136.000000,136.850006,136.850006,254221.0,136.600006,5074.600098,5080.689941,...,4992.190942,5118.492300,4865.889585,7731.375977,7984.059230,7478.692723,19809326.0,2.294370e+10,1.295351e+12,8.108808e+09
6,1708905600,135.500000,139.250000,135.050003,138.750000,138.750000,304159.0,136.850006,5093.000000,5097.660156,...,4984.678442,5107.263494,4862.093391,7717.829492,7949.717954,7485.941031,20063547.0,2.288397e+10,1.291425e+12,8.058411e+09
9,1708473600,135.300003,135.300003,132.649994,134.850006,134.850006,238055.0,134.250000,4963.029785,4983.209961,...,4955.094482,5063.497564,4846.691400,7664.382007,7830.537080,7498.226933,19747860.0,2.266789e+10,1.287385e+12,7.983094e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6207,947721600,36.000000,36.750000,34.000000,34.000000,19.221630,519684.0,37.400002,1432.250000,1454.199951,...,1431.084446,1478.000940,1384.167951,5614.952257,5887.180144,5342.724370,-890813.0,-3.567400e+06,2.500600e+09,0.000000e+00
6208,947635200,33.549999,35.139999,33.549999,34.889999,19.724779,186958.0,34.000000,1438.560059,1442.599976,...,1428.759995,1476.648518,1380.871471,5612.666260,5903.320486,5322.012034,-371129.0,-5.168360e+07,1.470200e+09,0.000000e+00
6209,947548800,35.189999,35.189999,33.750000,35.000000,19.786970,205261.0,34.889999,1457.599976,1458.660034,...,1428.261422,1479.897156,1376.625688,5616.975725,5929.811982,5304.139469,-184171.0,-8.636670e+07,2.444800e+09,0.000000e+00
6210,947462400,35.500000,35.490002,34.700001,35.200001,19.900040,201905.0,35.000000,1441.469971,1464.359985,...,1426.544983,1482.227186,1370.862780,5617.488363,5960.170428,5274.806297,21090.0,-4.247670e+07,3.458800e+09,0.000000e+00


In [10]:
# Set random seed for reproducibility
seed = 123

# Proportion for training set
train_prop = 0.8

# Split data into training and initial test sets
train_data, initial_test_data = train_test_split(Model_ML, test_size=1-train_prop, random_state=seed)

# Proportion for validation set (from the initial test set)
val_prop = 0.5

# Split initial test data into validation and testing sets
validate_data, test_data = train_test_split(initial_test_data, test_size=val_prop, random_state=seed)

# Reset row indices
train_data.reset_index(drop=True, inplace=True)
validate_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [11]:
# Feature Importance for Un-Normalized train_data dataframe
# Separate features and target variable
features = train_data.drop("THA_NextDay_Close", axis=1)  # Replace "target_column" with your target
target = train_data["THA_NextDay_Close"]

# Create the Random Forest model for regression
RFmodel = RandomForestRegressor(n_estimators=500)  # using 500 as there is a chance of overfitting n_estimators as needed

# Train the model
RFmodel.fit(features, target)

RandomForestRegressor(n_estimators=500)

In [12]:
# Get feature importances
importance = RFmodel.feature_importances_
feature_names = features.columns
sorted_idx = importance.argsort()[::-1]  # Sort features by importance (descending)
sorted_features = feature_names[sorted_idx]
sorted_importance = importance[sorted_idx]

# print sorted importance
print("Feature Importances (without scientific notation):\n")
for feature, importance in zip(sorted_features, sorted_importance):
    print(f"{feature}: {importance:.8f}") 

# Set a threshold for minimum importance (adjust as needed)
importance_threshold = 0.00001500

# Filter features and importance based on threshold
filtered_features = sorted_features[sorted_importance >= importance_threshold]
filtered_importance = sorted_importance[sorted_importance >= importance_threshold]

# Print or visualize importance
print("Sorted features by importance:\n", filtered_features)


Feature Importances (without scientific notation):

THA_Close: 0.90641491
THA_High: 0.06433100
THA_Adj_Close: 0.01812497
THA_Low: 0.00807312
THA_Open: 0.00096527
THA_MA_7: 0.00042215
FK_DT_Date: 0.00016517
THA_MA_90: 0.00015994
THA_UpperBand: 0.00014790
THA_MA_180: 0.00014516
THA_EMA26: 0.00010152
EUR_MA_180: 0.00008575
THA_SMA: 0.00003018
THA_EMA12: 0.00002949
THA_MA_30: 0.00002876
THA_MACD: 0.00002490
SPI_RSI_7: 0.00002457
SPI_RSI_90: 0.00002387
EUR_RSI_7: 0.00002346
THA_RSI_180: 0.00002238
THA_Volume: 0.00002189
EUR_Volume: 0.00002147
SPI_Volume: 0.00002112
THA_LowerBand: 0.00002085
FRA_RSI_7: 0.00001974
THA_RSI_30: 0.00001969
SPI_RSI_30: 0.00001818
SPI_MACD: 0.00001810
THA_RSI_7: 0.00001808
FRA_RSI_30: 0.00001704
EUR_RSI_90: 0.00001675
THA_OBV: 0.00001647
EUR_MACD: 0.00001590
SPI_RSI_180: 0.00001582
EUR_RSI_180: 0.00001452
EUR_RSI_30: 0.00001438
FRA_RSI_180: 0.00001415
THA_RSI_90: 0.00001377
FRA_MA_180: 0.00001340
FRA_RSI_90: 0.00001309
EUR_LowerBand: 0.00001302
FRA_Volume: 0.00001

In [13]:
# Sorted features by importance:
#  Index(['THA_Close', 'THA_High', 'THA_Adj_Close', 'THA_Low', 'THA_Open',
#        'THA_MA_7', 'FK_DT_Date', 'THA_UpperBand', 'THA_MA_90', 'THA_MA_180',
#        'THA_SMA', 'EUR_MA_180', 'THA_EMA12', 'SPI_MA_180', 'THA_EMA26',
#        'THA_MACD', 'SPI_RSI_7', 'EUR_RSI_7', 'THA_RSI_180', 'SPI_RSI_90',
#        'THA_Volume', 'EUR_Volume', 'SPI_Volume', 'THA_MA_30', 'FRA_RSI_7',
#        'THA_RSI_30', 'THA_LowerBand', 'THA_RSI_7', 'SPI_RSI_30', 'EUR_RSI_90',
#        'SPI_MACD', 'THA_OBV', 'FRA_RSI_30', 'EUR_MACD', 'SPI_RSI_180'],
#       dtype='object')

In [14]:
# Define your own `normalize()` function
def normalize(x):
    num = x - x.min()
    denom = x.max() - x.min()
    return num / denom

# Create a list of column names to exclude
exclude_columns = ["FK_DT_Date", "THA_NextDay_Close"]

# Normalize columns in the training dataset (excluding specified columns)
train_data.loc[:, ~train_data.columns.isin(exclude_columns)] = train_data.loc[:, ~train_data.columns.isin(exclude_columns)].apply(normalize)

# Normalize columns in the validation dataset (excluding specified columns)
validate_data.loc[:, ~validate_data.columns.isin(exclude_columns)] = validate_data.loc[:, ~validate_data.columns.isin(exclude_columns)].apply(normalize)

# Normalize columns in the testing dataset (excluding specified columns)
test_data.loc[:, ~test_data.columns.isin(exclude_columns)] = test_data.loc[:, ~test_data.columns.isin(exclude_columns)].apply(normalize)

In [15]:
# Feature Importance for Normalized train_data dataframe
# Separate features and target variable
features_Norm = train_data.drop("THA_NextDay_Close", axis=1)  
target_Norm = train_data["THA_NextDay_Close"]

# Create the Random Forest model
RFmodel_Norm = RandomForestRegressor(n_estimators=500)  # using 5400 as there is a chance of overfitting 

# Train the model
RFmodel_Norm.fit(features_Norm, target_Norm)

RandomForestRegressor(n_estimators=500)

In [16]:
# Feature Importance for Normalized train_data dataframe
# Get feature importances
importance_Norm = RFmodel_Norm.feature_importances_

# Get feature importances
importance_Norm = RFmodel_Norm.feature_importances_
feature_names_Norm = features_Norm.columns
sorted_idx_Norm = importance_Norm.argsort()[::-1]  # Sort features by importance (descending)
sorted_features_Norm = feature_names_Norm[sorted_idx_Norm]
sorted_importance_Norm = importance_Norm[sorted_idx_Norm]

# print sorted importance
print("Feature Importances (without scientific notation):\n")
for feature_Norm, importance_Norm in zip(sorted_features_Norm, sorted_importance_Norm):
    print(f"{feature_Norm}: {importance_Norm:.8f}") 

# Set a threshold for minimum importance (adjust as needed)
importance_threshold = 0.00001500

# Filter features and importance based on threshold
filtered_features_Norm = sorted_features_Norm[sorted_importance_Norm >= importance_threshold]
filtered_importance_Norm = sorted_importance_Norm[sorted_importance_Norm >= importance_threshold]

# Print or visualize importance
print("Sorted features by importance:\n", filtered_features_Norm)

Feature Importances (without scientific notation):

THA_Close: 0.90399750
THA_High: 0.06486019
THA_Adj_Close: 0.01861523
THA_Low: 0.00968184
THA_Open: 0.00074074
THA_MA_7: 0.00039540
FK_DT_Date: 0.00021613
THA_MA_180: 0.00017171
THA_UpperBand: 0.00016373
THA_MA_90: 0.00014759
THA_SMA: 0.00006973
THA_EMA26: 0.00005695
EUR_MA_180: 0.00003493
SPI_MA_180: 0.00003427
THA_EMA12: 0.00002989
THA_MACD: 0.00002477
THA_MA_30: 0.00002438
SPI_RSI_7: 0.00002422
EUR_RSI_7: 0.00002303
THA_RSI_180: 0.00002287
SPI_RSI_90: 0.00002228
SPI_Volume: 0.00002179
THA_Volume: 0.00002146
THA_LowerBand: 0.00002042
EUR_Volume: 0.00002031
FRA_RSI_7: 0.00002006
THA_RSI_30: 0.00001916
THA_RSI_7: 0.00001889
SPI_RSI_30: 0.00001812
SPI_MACD: 0.00001738
EUR_RSI_90: 0.00001652
FRA_RSI_30: 0.00001617
THA_OBV: 0.00001607
SPI_RSI_180: 0.00001597
EUR_MACD: 0.00001578
EUR_RSI_180: 0.00001506
FRA_RSI_180: 0.00001474
EUR_RSI_30: 0.00001402
THA_RSI_90: 0.00001309
FRA_Volume: 0.00001293
FRA_MA_180: 0.00001286
FRA_RSI_90: 0.00001250

In [17]:
# Random Forest importance > .00001500
# Sorted features by importance:
#  Index(['THA_Close', 'THA_High', 'THA_Adj_Close', 'THA_Low', 'THA_Open',
#        'THA_MA_7', 'FK_DT_Date', 'THA_UpperBand', 'THA_MA_90', 'THA_MA_180',
#        'THA_MA_30', 'EUR_MA_180', 'THA_SMA', 'THA_EMA12', 'SPI_MA_180',
#        'EUR_MA_90', 'THA_EMA26', 'EUR_MA_30', 'THA_MACD', 'SPI_RSI_7',
#        'EUR_RSI_7', 'THA_RSI_180', 'THA_Volume', 'THA_LowerBand', 'SPI_Volume',
#        'SPI_RSI_90', 'FRA_RSI_7', 'THA_RSI_30', 'EUR_Volume', 'SPI_RSI_30',
#        'THA_RSI_7', 'SPI_MACD', 'THA_OBV', 'EUR_RSI_90', 'SPI_RSI_180',
#        'FRA_RSI_30', 'EUR_MACD'],
#       dtype='object')

In [18]:
# Define features (X) and target (y)
X = train_data.drop(columns=['THA_NextDay_Close', 'FK_DT_Date'])
y = train_data['THA_NextDay_Close']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models (excluding ARIMA)
models = {
    'Random Forest': RandomForestRegressor(),
    'k-Nearest Neighbors': KNeighborsRegressor(),
    'Linear Regression': LinearRegression(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'Decision Tree': DecisionTreeRegressor(),
    'Multi-layer Perceptron': MLPRegressor(max_iter=1000),
    'XGBoost': XGBRegressor(),
    'Support Vector Machine (Poly)': SVR(kernel='poly'),
    'Support Vector Machine (Radial)': SVR(kernel='rbf'),
    'Support Vector Machine (Linear)': SVR(kernel='linear')
}

# Function to calculate and print evaluation metrics
def evaluate_model(model_name, model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)

    print(f"{model_name}: MSE = {mse:.4f}, RMSE = {rmse:.4f}, MAE = {mae:.4f}")


# Loop through models, train, and evaluate
for model_name, model in models.items():
    model.fit(X_train, y_train)
    evaluate_model(model_name, model, X_test, y_test)

Random Forest: MSE = 1.7481, RMSE = 1.3222, MAE = 0.7884
k-Nearest Neighbors: MSE = 1.9726, RMSE = 1.4045, MAE = 0.9244
Linear Regression: MSE = 1.6161, RMSE = 1.2713, MAE = 0.7594
Gradient Boosting: MSE = 1.7474, RMSE = 1.3219, MAE = 0.8038
Decision Tree: MSE = 2.8045, RMSE = 1.6747, MAE = 1.0356
Multi-layer Perceptron: MSE = 2.1022, RMSE = 1.4499, MAE = 0.9098
XGBoost: MSE = 2.0275, RMSE = 1.4239, MAE = 0.8628
Support Vector Machine (Poly): MSE = 3.0388, RMSE = 1.7432, MAE = 1.1928
Support Vector Machine (Radial): MSE = 5.2402, RMSE = 2.2892, MAE = 1.3272
Support Vector Machine (Linear): MSE = 2.1323, RMSE = 1.4602, MAE = 0.8534


In [19]:
# Random Forest: MSE = 1.7470, RMSE = 1.3217, MAE = 0.7849
# k-Nearest Neighbors: MSE = 1.9726, RMSE = 1.4045, MAE = 0.9244
# Linear Regression: MSE = 1.6161, RMSE = 1.2713, MAE = 0.7594
# Gradient Boosting: MSE = 1.7397, RMSE = 1.3190, MAE = 0.8029
# Decision Tree: MSE = 2.9266, RMSE = 1.7107, MAE = 1.0505
# Multi-layer Perceptron: MSE = 2.1505, RMSE = 1.4665, MAE = 0.9021
# XGBoost: MSE = 2.0275, RMSE = 1.4239, MAE = 0.8628
# Support Vector Machine (Poly): MSE = 3.0388, RMSE = 1.7432, MAE = 1.1928
# Support Vector Machine (Radial): MSE = 5.2402, RMSE = 2.2892, MAE = 1.3272
# Support Vector Machine (Linear): MSE = 2.1323, RMSE = 1.4602, MAE = 0.8534

In [20]:
# Define features (X) for model training using selected features with RF importance > .00001500
X = train_data[filtered_features_Norm]

# Define target variable (y) for model training and evaluation
y = train_data['THA_NextDay_Close']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models (excluding ARIMA)
models = {
    'Random Forest': RandomForestRegressor(),
    'k-Nearest Neighbors': KNeighborsRegressor(),
    'Linear Regression': LinearRegression(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'Decision Tree': DecisionTreeRegressor(),
    'Multi-layer Perceptron': MLPRegressor(max_iter=1000),
    'XGBoost': XGBRegressor(),
    'Support Vector Machine (Poly)': SVR(kernel='poly'),
    'Support Vector Machine (Radial)': SVR(kernel='rbf'),
    # 'Support Vector Machine (Linear)': SVR(kernel='linear')
}

# Function to calculate and print evaluation metrics
def evaluate_model(model_name, model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)

    print(f"{model_name}: MSE = {mse:.4f}, RMSE = {rmse:.4f}, MAE = {mae:.4f}")


# Loop through models, train, and evaluate
for model_name, model in models.items():
    model.fit(X_train, y_train)
    evaluate_model(model_name, model, X_test, y_test)

Random Forest: MSE = 1.6974, RMSE = 1.3028, MAE = 0.7830
k-Nearest Neighbors: MSE = 1.6675, RMSE = 1.2913, MAE = 0.8154
Linear Regression: MSE = 1.6326, RMSE = 1.2777, MAE = 0.7589
Gradient Boosting: MSE = 1.7363, RMSE = 1.3177, MAE = 0.8059
Decision Tree: MSE = 2.4769, RMSE = 1.5738, MAE = 1.0240
Multi-layer Perceptron: MSE = 83679.0447, RMSE = 289.2733, MAE = 286.7356
XGBoost: MSE = 2.0169, RMSE = 1.4202, MAE = 0.8589
Support Vector Machine (Poly): MSE = 314.3988, RMSE = 17.7313, MAE = 14.8374
Support Vector Machine (Radial): MSE = 371.4083, RMSE = 19.2720, MAE = 15.3392


In [21]:
# Random Forest: MSE = 1.7165, RMSE = 1.3101, MAE = 0.7868
# k-Nearest Neighbors: MSE = 1.6675, RMSE = 1.2913, MAE = 0.8154
# Linear Regression: MSE = 1.6353, RMSE = 1.2788, MAE = 0.7589
# Gradient Boosting: MSE = 1.7321, RMSE = 1.3161, MAE = 0.8050
# Decision Tree: MSE = 2.5630, RMSE = 1.6010, MAE = 1.0341
# Multi-layer Perceptron: MSE = 60057.2103, RMSE = 245.0657, MAE = 237.6592
# XGBoost: MSE = 2.0459, RMSE = 1.4304, MAE = 0.8419
# Support Vector Machine (Poly): MSE = 314.3988, RMSE = 17.7313, MAE = 14.8374
# Support Vector Machine (Radial): MSE = 371.4083, RMSE = 19.2720, MAE = 15.3392

In [22]:


X = train_data.drop('THA_NextDay_Close', axis=1)
y = train_data['THA_NextDay_Close']

# Impute missing values using KNN
imputer = KNNImputer()
X_imputed = imputer.fit_transform(X)

# Create the Random Forest model
modelRF = RandomForestRegressor()

# Train the model
modelRF.fit(X_imputed, y)

# Evaluate on training data (use with caution)
evaluate_model("Random Forest (KNN Imputation)", modelRF, X_imputed, y)

Random Forest (KNN Imputation): MSE = 0.1785, RMSE = 0.4225, MAE = 0.2653


In [ ]:
# Random Forest (KNN Imputation): MSE = 0.1785, RMSE = 0.4225, MAE = 0.2653

In [23]:
# # confusion matrix???
# from sklearn.metrics import confusion_matrix

# print("Confusion Matrix:\n", confusion_matrix)

# # Assuming you have a trained model 'model' and true labels 'y_true'
# y_pred = model.predict(X_test)  # Get predictions for test data

# confusion_matrix = confusion_matrix(y_true, y_pred)
# # Or for visualization with matplotlib:
# plt.figure(figsize=(6, 6))
# plt.imshow(confusion_matrix, cmap=plt.cm.Blues)
# plt.title("Confusion Matrix")
# plt.xticks(range(len(data.target_names)), data.target_names, rotation=45)
# plt.yticks(range(len(data.target_names)), data.target_names)
# plt.colorbar()
# plt.show()
